# Illustrations

In [1]:
from dhlab.module_update import update, css
update('nbpictures')
import json
import requests
import dhlab.nbpictures as nbp
import dhlab.nbtext as nb
from PIL import Image
from IPython.display import HTML
import re
%matplotlib inline

Updated file `C:\Users\larsj\Documents\GitHub\illustrations_bokhylla\nbpictures.py`

In [61]:
update('nb_elastic_collocations')

Updated file `C:\Users\larsj\Documents\GitHub\illustrations_bokhylla\nb_elastic_collocations.py`

In [62]:
import nb_elastic_collocations as nec

In [105]:
def get_illustrations(urn):
    import re
    if 'digibok' in str(urn):
        urn = re.findall("[0-9]{13}", str(urn))[0] 
    r = requests.get('https://api.nb.no/ngram/illustrations', json={'urn': urn})
    return r.json() 

## Kode for å finne bilder og manipulere dem

In [52]:
small_scale = 0.59
large_scale = 1.58

def iiif_manifest(urn):
    if not 'URN' in str(urn) and not 'digibok' in str(urn):
        urn = "URN:NBN:no-nb_digibok_" + str(urn)
    elif not 'URN' in str(urn):
        urn = "URN:NBN:no-nb_" + str(urn)
    r = requests.get("https://api.nb.no/catalog/v1/iiif/{urn}/manifest".format(urn=urn))
    return r.json()


    
def load_picture(url):
    r = requests.get(url, stream=True)
    r.raw.decode_content=True
    #print(r.status_code)
    return r.raw

def metadata(ill):
    #print(ill)
    u = bookurn_from_illustration(get_url_from_illustration(ill, part=100))
    m = iiif_manifest(u)['metadata']
    return m

def meta2html(ill):
    d = metadata(ill)
    ls = ""
    for i in d:
        ls += "<dt>{label}</dt><dd>{value}</dd>".format(label=i['label'], value=i['value'])
    return "<dl>{ls}</dl>".format(ls=ls)

def bookurn_from_illustration(ill):
    import re
    #print(ill)
    hits = re.findall("URN[^/]+", ill)
    if hits != []:
        res = hits[0].split('_')[:-1]
        res = '_'.join(res)
    return res

def get_url_from_illustration(illus, part = 200, scale = small_scale):
    urn = "URN:NBN:no-nb_" + illus['page']
    #print(urn)
    if part == True:
        # return cut out
        url = "https://www.nb.no/services/image/resolver/{urn}/{hpos},{vpos},{width},{height}/full/0/native.jpg".format(
            urn = urn, 
            width = int(int(illus['width'])*scale), 
            height = int(int(illus['height'])*scale), 
            vpos = int(int(illus['vpos'])*scale), 
            hpos = int(int(illus['hpos'])*scale)
        )
    else:
        # return whole page
        url = "https://www.nb.no/services/image/resolver/{urn}/full/0,{part}/0/native.jpg".format( part=part,
                urn = urn, width=illus['width'], height= illus['height'], vpos=illus['vpos'], hpos=illus['hpos'])
    
    return url

def get_url(illus, part = True, scale = None, cuts = True):
    """part sets size of output of page, if part is True it returns the cut out of image
    illus is a dictionary of with entries and values like this: 
    {'height': 270, 'hpos': 251, 'page': 'digibok_2017081626006_0018', 'resolution': 400, 'vpos': 791, 'width': 373} """
    
    if scale == None:
        if illus['resolution'] >= 300 or illus['resolution'] < 100:
            scale = large_scale
        else:
            scale = small_scale
            
    height = illus['height']
    width = illus['width']

    if cuts != False:
        if width * scale > 1024:
            width = int(1024/scale)
        if height * scale > 1024:
            height = int(1024/scale)
            
    #scale = int(illus['resolution'])/250
    urn = "URN:NBN:no-nb_" + illus['page']
    #print(urn)
    if part == True:
        # return cut out
        url = "https://www.nb.no/services/image/resolver/{urn}/{hpos},{vpos},{width},{height}/full/0/native.jpg".format(
            urn = urn, 
            width = int(width * scale), 
            height = int(height * scale), 
            vpos = int(int(illus['vpos']) * scale), 
            hpos = int(int(illus['hpos']) * scale)
        )
    else:
        # return whole page
        url = "https://www.nb.no/services/image/resolver/{urn}/full/0,{part}/0/native.jpg".format( part=part,
                urn = urn, width=illus['width'], height= illus['height'], vpos=illus['vpos'], hpos=illus['hpos'])
    
    return url

def get_picture_from_illustration(illus, page=True, scale=small_scale):
    if page == True:
        url = get_url_from_illustration(illus, part=500)
    else:
        url = get_url_from_illustration(illus, part = True, scale=scale)
    return Image.open(load_picture(url))

def get_picture(illus, page=True, scale = small_scale):
    #if int(illus['resolution']) >= 400:
    #    scale = large_scale
    scale = int(illust['resolution'])/250
    if page == True:
        url = get_url_from_illustration(illus, part=500)
    else:
        url = get_url_from_illustration(illus, part = True, scale=scale)
    return Image.open(load_picture(url))

def display_finds(r):
    """A list of urls in r is displayed as HTML"""
    rows = ["<tr><td><img src='{row}'</td><td><a href = {meta} target='_'>{meta}</a></td></tr>".format(row=row, meta=row) for row in r]
    return HTML("""<html><head></head>
     <body>
     <table>
     {rows}
     </table>
     </body>
     </html>
     """.format(rows=' '.join(rows)))

def url2urn(url):
    return re.findall("URN:.*[0-9]{13}", url)[0]

In [4]:
url2urn("https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0018/396,1249,589,426/full/0/native.jpg")

'URN:NBN:no-nb_digibok_2017081626006'

In [125]:
def urns_from_super(term):
    res = nec.super_search(term, mediatype = 'bøker')
    ids = [x['metadata']['identifiers']  for x in res['_embedded']['items'] ]
    return [x['urn'] for x in ids if 'urn' in x]

In [129]:
bok = urns_from_super('folkemusikk')

In [130]:
bok

['URN:NBN:no-nb_digibok_2012042508000',
 'URN:NBN:no-nb_digibok_2016090948114',
 'URN:NBN:no-nb_digibok_2014091008187',
 'URN:NBN:no-nb_digibok_2012080808132',
 'URN:NBN:no-nb_digibok_2012101206047',
 'URN:NBN:no-nb_digibok_2013102308124',
 'URN:NBN:no-nb_digibok_2014013008111',
 'URN:NBN:no-nb_digibok_2014080806105',
 'URN:NBN:no-nb_digibok_2017021548069',
 'URN:NBN:no-nb_pliktavlevering_7834',
 'URN:NBN:no-nb_digibok_2011100608119',
 'URN:NBN:no-nb_digibok_2015061708183',
 'URN:NBN:no-nb_digibok_2010052003016',
 'URN:NBN:no-nb_digibok_2015073008099',
 'URN:NBN:no-nb_digibok_2014120108015',
 'URN:NBN:no-nb_digibok_2012101507074',
 'URN:NBN:no-nb_digibok_2007072601070',
 'URN:NBN:no-nb_digibok_2008121001017',
 'URN:NBN:no-nb_digibok_2009082100046',
 'URN:NBN:no-nb_digibok_2017110748181',
 'URN:NBN:no-nb_digibok_2015111008076',
 'URN:NBN:no-nb_digibok_2010121706024',
 'URN:NBN:no-nb_digibok_2008042804039',
 'URN:NBN:no-nb_digibok_2011102508076',
 'URN:NBN:no-nb_digibok_2020030665019',
 

In [131]:
illus = get_illustrations(bok[0])
urls = [get_url(ill, part = True, cuts = True) for ill in illus]
display_finds(urls)

"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2012042508000_0001/187,2975,1023,521/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2012042508000_0001/187,2975,1023,521/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2012042508000_0002/384,213,1023,1023/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2012042508000_0002/384,213,1023,1023/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2012042508000_0002/1720,1677,1023,998/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2012042508000_0002/1720,1677,1023,998/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2012042508000_0002/1785,3576,1023,500/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2012042508000_0002/1785,3576,1023,500/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2012042508000_0003/109,225,1023,1023/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2012042508000_0003/109,225,1023,1023/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2012042508000_0003/73,1786,1023,152/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2012042508000_0003/73,1786,1023,152/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2012042508000_0004/356,2198,1023,1023/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2012042508000_0004/356,2198,1023,1023/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2012042508000_0005/0,2303,1023,1023/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2012042508000_0005/0,2303,1023,1023/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2012042508000_0005/1506,2286,1023,1023/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2012042508000_0005/1506,2286,1023,1023/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2012042508000_0007/34,47,601,541/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2012042508000_0007/34,47,601,541/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2012042508000_0008/2424,4374,613,100/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2012042508000_0008/2424,4374,613,100/full/0/native.jpg"


In [56]:
iiif_manifest(2016090708313)

{'@context': 'http://iiif.io/api/presentation/2/context.json',
 '@type': 'sc:Manifest',
 '@id': 'https://api.nb.no/catalog/v1/iiif/URN:NBN:no-nb_digibok_2016090708313/manifest{?fields,profile}',
 'label': 'Mobilisering, utvikling og læring : evaluering av prosessen rundt forsøksordningen for utvikling av kollektivtransport',
 'metadata': [{'label': 'Tilgang',
   'value': "Tilgjengelig etter bestemte vilkår (<a href='https://www.nb.no/hjelp-og-informasjon/rettigheter/'>Les om tilgangsvilkår</a>)"},
  {'label': 'Tittel',
   'value': 'Mobilisering, utvikling og læring : evaluering av prosessen rundt forsøksordningen for utvikling av kollektivtransport'},
  {'label': 'Forfatter', 'value': 'Stenstadvold, Morten'},
  {'label': 'Publisert', 'value': 'Oslo:Transportøkonomisk institutt, 1993'},
  {'label': 'Emne', 'value': 'Kollektivtrafikk'},
  {'label': 'Språk', 'value': 'Norsk (Bokmål)'},
  {'label': 'ISBN', 'value': '8271338285'},
  {'label': 'Kilde for metadata',
   'value': 'nb.bibsys.no 

In [54]:
illus = get_illustrations(2016090708313)
urls = [get_url(ill, part = True, cuts = True) for ill in illus]
display_finds(urls)

"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0018/396,1249,589,426/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0018/396,1249,589,426/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0019/543,755,265,504/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0019/543,755,265,504/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0020/379,589,507,440/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0020/379,589,507,440/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0021/180,237,959,518/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0021/180,237,959,518/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0022/274,510,755,372/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0022/274,510,755,372/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0023/372,401,480,590/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0023/372,401,480,590/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0025/357,797,627,704/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0025/357,797,627,704/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0026/257,481,780,594/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0026/257,481,780,594/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0027/252,254,807,535/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0027/252,254,807,535/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0027/292,1113,782,583/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0027/292,1113,782,583/full/0/native.jpg"
"<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0028/216,578,878,649/full/0/native.jpg'","https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2017081626006_0028/216,578,878,649/full/0/native.jpg"


In [10]:
b = nb.book_urn(period = (1800, 1820))

In [13]:
b

[[2013080124001, 'Novalis', 'Novalis Schriften', 1802],
 [2012071213005, 'Wulfsberg, N.', 'Sang for Kronprindsens Livcorps', 1801],
 [2012041113001,
  'Wahlström, Per',
  'blick på Norriges närvarande belägenhet och interesse, samt hela den Skandinaviska half-öns derpå grundade väntan och hopp',
  1814],
 [2013070824014,
  'Cook, James',
  'Premier voyage de James Cook, autour du monde, fait en 1768, 1769, 1770 et 1771, précédé des relations de MM. Byron, Carteret, et Wallis',
  1804],
 [2016091426001,
  'Ancillon, Frédéric',
  'Om Souverainitet og Statsforfatninger',
  1820],
 [2009061013013,
  '',
  'Kundgjørelse angaaende et under Regentskabet etableret Regjeringsraaad [sic]',
  1814],
 [2010010803007,
  'Eggers, C.U.D. von',
  'Entwurf eines peinlichen Gesetzbuchs für die Herzogthümer Schleswig und Holstein',
  1808],
 [2012010713001,
  'Rodtwitt, Lauritz Augustinus',
  'Prædiken paa Fastelavens Søndag, holden til Høymesse i St. Jørgens Hospitals Kirke, i Anledning af Ildebranden i

In [15]:
b[0][0]

2013080124001

In [16]:
iiif_manifest(b[0][0])

{'@context': 'http://iiif.io/api/presentation/2/context.json',
 '@type': 'sc:Manifest',
 '@id': 'https://api.nb.no/catalog/v1/iiif/URN:NBN:no-nb_digibok_2013080124001/manifest{?fields,profile}',
 'label': 'Novalis Schriften. T. 1',
 'metadata': [{'label': 'Tilgang', 'value': 'Tilgang for alle'},
  {'label': 'Tittel', 'value': 'Novalis Schriften. T. 1'},
  {'label': 'Forfatter', 'value': 'Novalis'},
  {'label': 'Medforfatter/Bidragsyter',
   'value': 'Bülow, Eduard von | Schlegel, Friedrich | Tieck, Ludwig'},
  {'label': 'Publisert',
   'value': 'Berlin:In der Buchhandlung der Realschule, 1802'},
  {'label': 'Andre opplysninger',
   'value': 'Elektronisk reproduksjon [Norge] Nasjonalbiblioteket Digital 2013-08-08\n'},
  {'label': 'Språk', 'value': 'Tysk'},
  {'label': 'Kilde for metadata',
   'value': 'nb.bibsys.no (<a title="Link til post i Oria" href="https://www.oria.no/?vid=NB&search=999720901344702202">999720901344702202</a>)'},
  {'label': 'Omfang', 'value': '338 s. 15 cm'},
  {'l

In [18]:
display_finds(['https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2013080124001_0003/full/full/0/native.jpg'])

<img src='https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2013080124001_0003/full/full/0/native.jpg',https://www.nb.no/services/image/resolver/URN:NBN:no-nb_digibok_2013080124001_0003/full/full/0/native.jpg
